In [1]:
%matplotlib ipympl

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import gsw
from scipy.special import ker,kei

In [2]:
dstide = xr.open_dataset('utide_results.nc')

In [3]:
i = 1j
ap = 0.5*(dstide['Lsmaj']+dstide['Lsmin'])*np.exp(i*(dstide['theta']-dstide['g'])*np.pi/180)
am = 0.5*(dstide['Lsmaj']-dstide['Lsmin'])*np.exp(i*(dstide['theta']+dstide['g'])*np.pi/180)
Ap = np.abs(ap)
Am = np.abs(am)
epsilonp = np.angle(ap)*180/np.pi
epsilonm = np.angle(am)*180/np.pi

In [4]:
dims = ('height', 'constituent')
dstide['ap'] = (dims, ap)
dstide['am'] = (dims, am)
dstide['Ap'] = (dims, Ap)
dstide['Am'] = (dims, Am)
dstide['epsilonp'] = (dims, epsilonp)
dstide['epsilonm'] = (dims, epsilonm)

| Parameter                    | Soulsby (theory) | Codiga (UTide)           |
|------------------------------|------------------|--------------------------|
| major axis                   | $U_a$            | $L^{smaj}_q$             |
| minor axis                   | $U_b$            | $L^{smin}_q$             |
| complex anticlockwise vector | $R_+$            | $a^+_q$                  |
| complex clockwise vector     | $R_-$            | $a^-_q$                  |
| anticlockwise amplitude      | $|R_+|$          | $A^+_q$                  |
| clockwise amplitude          | $|R_-|$          | $A^-_q$                  |
| orientation angle            | $\Phi$           | $\theta_q$               |
| phase lag                    | $\phi$           | $g_q$                    |
| anticlockwise phase          | $\phi_+$         | $\epsilon^+_q$, -$g^+_q$ |
| clockwise phase              | $\phi_-$         | $\epsilon^-_q$, $g^-_q$  |

### Plot M2 parameters

In [5]:
zi = np.arange(0,len(dstide['height'])-7)
ci, = np.where(dstide['constituent'] == 'K1')

### Define model equations

In [13]:
def planetary_bbl_structure(ustar,zo,z,f):
    '''
    Calculate vertical structure of a planetary boundary layer (a.k.a.
    bottom Ekman layer) following Soulsby (1983). The calculation assumes
    that the eddy viscosity increases linearly with height.
    '''

    i = 1j 
    kappa = 0.41 # Von Karman's constant
    
    # non-dimensional height and roughness length
    xi = 2*np.sqrt(f*z/(kappa*ustar))
    xio = 2*np.sqrt(f*zo/(kappa*ustar))
    
    # Q/Qinf in Equation 31 of Soulsby (1983)
    fac = (1-(ker(xi)*ker(xio) + kei(xi)*kei(xio))/
             (ker(xio)**2 + kei(xio)**2) + 
           i*(ker(xi)*kei(xio) - kei(xi)*ker(xio))/
             (ker(xio)**2 + kei(xio)**2))

    return fac

def rotating_planetary_bbl_coeffs(ustar,zo,Rpinf,Rminf,z,sigma,f,zoffset=0):
    if sigma+f < 0:
        Rpinf = np.abs(Rpinf)*np.exp(-i*np.angle(Rpinf))
    if sigma-f > 0:
        Rminf = np.abs(Rminf)*np.exp(-i*np.angle(Rminf))

    Rp = Rpinf*planetary_bbl_structure(ustar,zo,z,sigma+f)
    Rm = Rminf*planetary_bbl_structure(ustar,zo,z,sigma-f)

    if sigma+f < 0:
        Rp = Rpinf*planetary_bbl_structure(ustar,zo,z,-(sigma+f))
    if sigma-f < 0:
        Rm = Rminf*planetary_bbl_structure(ustar,zo,z,-(sigma-f))
    
    phip = np.angle(Rp)
    phim = np.angle(Rm)

    if sigma+f < 0:
        # southern hemisphere (needs testing) 
        # tidal oscillation lower frequency than f
        Rp = Rpinf*planetary_bbl_structure(ustar,zo,z,sigma+f)
        phip = -np.angle(Rp)
        Rp = np.abs(Rp)*np.exp(1j*phip)

    if sigma-f > 0:
        # northern hemisphere 
        # tidal oscillation higher frequency than f
        phim = -np.angle(Rm)
        Rm = np.abs(Rm)*np.exp(1j*phim)
        
    return Rp,Rm

In [14]:
print('Rpinf:',np.array(ap[zi[-1],ci]))
print('Rminf:',np.array(am[zi[-1],ci]))

Rpinf: [0.0019333+0.0065062j]
Rminf: [-0.00240394-0.00588596j]


In [15]:
Rpinf = np.array(ap[zi[-1],ci])
Rminf = np.array(am[zi[-1],ci])
ustar = 0.001
zo = 0.01

z = np.arange(0.05,25,0.05)
sigma = float(2*np.pi*dstide['freq'][ci]/3600)
f = gsw.f(35+8.4585/60)

In [16]:
Rp,Rm = rotating_planetary_bbl_coeffs(ustar,zo,Rpinf,Rminf,z,sigma,f)

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: invalid value encountered in sqrt
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:13: RuntimeWarning: invalid value encountered in sqrt


In [21]:
import lmfit

def fit_function(params, z=None, dat_ccw=None, dat_cw=None):
    
    Rp,Rm = rotating_planetary_bbl_coeffs(params['ustar'],
                                          params['zo'],
                                          params['Rpinf_real']+1j*params['Rpinf_imag'],
                                          params['Rminf_real']+1j*params['Rminf_imag'],
                                          z+params['z_offset'],
                                          params['sigma'],
                                          params['f'])

    resid1 = np.abs(np.real(dat_ccw) - np.real(Rp))
    resid2 = np.abs(np.imag(dat_ccw) - np.imag(Rp))
    resid3 = np.abs(np.real(dat_cw) - np.real(Rm))
    resid4 = np.abs(np.imag(dat_cw) - np.imag(Rm))    
    
    return np.concatenate((resid1, resid2, resid3, resid4))

params = lmfit.Parameters()
params.add('f', value=f, vary=False)
params.add('sigma', value=sigma, vary=False)
params.add('zo', value=0.001)
params.add('ustar', value=ustar)
params.add('Rpinf_real', value=np.real(Rpinf),vary=True)
params.add('Rpinf_imag', value=np.imag(Rpinf),vary=True)
params.add('Rminf_real', value=np.real(Rminf),vary=True)
params.add('Rminf_imag', value=np.imag(Rminf),vary=True)
params.add('z_offset', value=0,vary=False)  # acccount for unknowns in measurement levels

dat_z = np.array(dstide['height'][zi])
dat_ccw = np.array(ap[zi,ci]).flatten()
dat_cw = np.array(am[zi,ci]).flatten()

out_c = lmfit.minimize(fit_function, params, args=(dat_z, dat_ccw, dat_cw))

In [22]:
z = np.arange(0.1,20,0.01)

zo_fit = float(out_c.params['zo'])
ustar_fit = float(out_c.params['ustar'])
Rpinf_fit = float(out_c.params['Rpinf_real'])+1j*float(out_m2.params['Rpinf_imag'])
Rminf_fit = float(out_c.params['Rminf_real'])+1j*float(out_m2.params['Rminf_imag'])

Rp,Rm = rotating_planetary_bbl_coeffs(ustar_fit,zo_fit,Rpinf_fit,Rminf_fit,z,sigma,f)

print('zo',zo_fit)
print('ustar',ustar_fit)

zo 0.11231131346370697
ustar 0.0012063341006544636


/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: invalid value encountered in sqrt
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:13: RuntimeWarning: invalid value encountered in sqrt


In [23]:
out_m2.params['zo']

<Parameter 'zo', value=0.11231131346370697 +/- 0.0173, bounds=[-inf:inf]>

### Station M 

The model will be fit to the complex rotary coefficients calculated from the UTide results. The free parameters should include:

* Anticlockwise coefficients above BBL: $\text{Re}(R_+^{\infty})$, $\text{Im}(R_+^{\infty})$
* Clockwise coefficients above BBL: $\text{Re}(R_-^{\infty})$, $\text{Im}(R_-^{\infty})$
* Maximum shear velocity ($u_{*m}$)
* Roughness length ($z_o$)

This is a total of six parameters. The coefficients above the BBL could also potentially be treated as known paramaters, using the top bin from the data.

The M2 constituent is the strongest signal. Can other constituents be used? Will the value of $z_o$ change?

In [25]:
phip = np.angle(Rp)
phim = np.angle(Rm)

phi = (phim-phip)/2
PHI = (phim+phip)/2

Ua = np.abs(Rp) + np.abs(Rm)
Ub = np.abs(Rp) - np.abs(Rm)

In [31]:
plt.figure(figsize=(10,5))
plt.subplot(131)
plt.plot(dstide['Lsmaj'][zi,ci],dstide['height'][zi],'ko')
plt.plot(dstide['Lsmin'][zi,ci],dstide['height'][zi],'o',color='gray')
plt.plot(Ua,z,'k')
plt.plot(Ub,z,color='gray')
plt.legend(['major - obs','minor - obs','major - model fit','minor - model fit'])
plt.title('$K_1$ major/minor axes')
plt.ylabel('height [m]')
plt.xlabel('[m/s]')
plt.ylim([0,20])
xl = plt.xlim()
yl = plt.ylim()
plt.text(xl[0],yl[1]+0.02*np.diff(yl),'a)')

plt.subplot(133)
plt.plot(PHI*180/np.pi+180,z,'k')
plt.plot(dstide['theta'][:,ci][zi],dstide['height'][zi],'ks')
#plt.legend(['$g$, $\phi$','$\\theta$, $\Phi$'])
plt.title('phase of rotary coefficients')
plt.title('$K_1$ orientation angle')
plt.gca().set_yticklabels([])
plt.xlabel('[degrees]')
plt.ylim([0,20])
xl = plt.xlim()
yl = plt.ylim()
plt.text(xl[0],yl[1]+0.02*np.diff(yl),'c)')

plt.subplot(132)
plt.plot(dstide['g'][:,ci][zi],dstide['height'][zi],'ks')
# add 180 degrees?
plt.plot(phi[2:]*180/np.pi+180,z[2:],'k')
plt.title('$K_1$ phase lag')
plt.gca().set_yticklabels([])
plt.xlabel('[degrees]')
plt.legend(['obs','model fit'])
plt.ylim([0,20])
xl = plt.xlim()
yl = plt.ylim()
plt.text(xl[0],yl[1]+0.02*np.diff(yl),'b)')

plt.savefig('figures_paper/tidal_model_fit_K1.png',dpi=600)
plt.savefig('figures_paper/tidal_model_fit_K1.pdf')

FigureCanvasNbAgg()